In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import locale
import os
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' )

'en_US.UTF-8'

In [2]:
#our handpicked initial columns that we want to look at


desired_col=np.array(['id','neighbourhood_cleansed','neighbourhood_group_cleansed', 'property_type', 'room_type', 'accommodates','bathrooms', 'bedrooms', 'beds', 'bed_type', 'price','cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights', 'availability_90','availability_365','review_scores_rating', 'review_scores_accuracy','review_scores_location', 'review_scores_value', 'calculated_host_listings_count', 'reviews_per_month', 'longitude', 'latitude'])


In [31]:
#add in replace nan with 0 for cleaning fee 
def read_in_airbnb_file_pair(files, desired_col):
    both_yrs = []
    to_replace = {"Brooklyn" :"Kings", "Manhattan":"New York", "Bedford-Stuyvesant": "Bedford Stuyvesant", "Chinatown":"Lower East Side", "Battery Park City":"Battery Park", 'Ditmars Steinway' : 'Astoria','Downtown Brooklyn' : 'Brooklyn Heights','Hell\'s Kitchen' : 'Midtown','Kips Bay' : 'Midtown', 'Ozone Park' : 'south ozone park','Prospect-Lefferts Gardens' : 'prospect lefferts gardens','South Slope' : 'park slope','Theater District' : 'midtown', 'Columbia St' : 'lower east side'}
    for file in files:
        yr = pd.read_csv('./Data/NYC_full_listings/'+file).replace(to_replace)
        #print(yr.columns)
        #print(list(set(yr["neighbourhood_cleansed"])))
        yr = yr.dropna(subset = ['id','neighbourhood_cleansed','bedrooms', 'accommodates','price', 'guests_included','extra_people', 'minimum_nights'])
        if 'cleaning_fee' not in yr.columns:
            yr['cleaning_fee'] = ["$0.00"]
        yr = yr.fillna(value= {'cleaning_fee':'$0.00'})
        yr = fix_columns(yr.loc[:, desired_col], desired_col)
        yr = yr.loc[yr["room_type"] == "Entire home/apt"]
        both_yrs.append(yr)
    return both_yrs[1], both_yrs[0]

In [37]:
## Split airbnb files names by month/year

airbnb_files = sorted(os.listdir('./Data/NYC_full_listings'))
airbnb_files_all = airbnb_files.remove(".DS_Store")                                ###[MHSJ]
airbnb_files_monthyear = []
for i in range(len(airbnb_files)):
    if airbnb_files[i] == ".DS_Store" or airbnb_files[i] == ".ipynb_checkpoints":  ###[MHSJ]
        continue                                                                   ###[MHSJ]
    year = int(airbnb_files[i].split('_')[0])
    month = int(airbnb_files[i].split('_')[1][:-4])         
    airbnb_files_monthyear.append((month, year))

In [38]:
#get airbnb date pairs for 1 year difference 

airbnb_files_datePairs = []
for i,date in enumerate(airbnb_files_monthyear):
    if (date[0], date[1]+1) in airbnb_files_monthyear:
        index_next_yr = airbnb_files_monthyear.index((date[0], date[1]+1))
        airbnb_files_datePairs.append((airbnb_files[i], airbnb_files[index_next_yr]))
print(len(airbnb_files_datePairs))
print(airbnb_files_datePairs[0:5])

27
[('2015_01.csv', '2016_01.csv'), ('2015_05.csv', '2016_05.csv'), ('2015_06.csv', '2016_06.csv'), ('2015_08.csv', '2016_08.csv'), ('2015_09.csv', '2016_09.csv')]


In [39]:
###[mhsj] get airbnb date pairs for < 1 year difference 

def get_date_pairs(lag, files, monthyear_files):
    airbnb_files_datePairs = []
    for i,date in enumerate(monthyear_files):
        if date[0]+lag > 12:
            (M,Y) = ((date[0]+lag)%12, date[1]+1)
        else:
            (M,Y) = (date[0] + lag, date[1])
        if (M,Y) in monthyear_files:
            index_next_yr = monthyear_files.index((M,Y))
            airbnb_files_datePairs.append((files[i], files[index_next_yr]))
    return airbnb_files_datePairs

lag_pairs = get_date_pairs(3, airbnb_files, airbnb_files_monthyear)
print(len(lag_pairs))
print(lag_pairs)

35
[('2015_03.csv', '2015_06.csv'), ('2015_05.csv', '2015_08.csv'), ('2015_06.csv', '2015_09.csv'), ('2015_08.csv', '2015_11.csv'), ('2015_09.csv', '2015_12.csv'), ('2015_10.csv', '2016_01.csv'), ('2015_11.csv', '2016_02.csv'), ('2016_01.csv', '2016_04.csv'), ('2016_02.csv', '2016_05.csv'), ('2016_04.csv', '2016_07.csv'), ('2016_05.csv', '2016_08.csv'), ('2016_06.csv', '2016_09.csv'), ('2016_07.csv', '2016_10.csv'), ('2016_08.csv', '2016_11.csv'), ('2016_09.csv', '2016_12.csv'), ('2016_10.csv', '2017_01.csv'), ('2016_11.csv', '2017_02.csv'), ('2016_12.csv', '2017_03.csv'), ('2017_01.csv', '2017_04.csv'), ('2017_02.csv', '2017_05.csv'), ('2017_03.csv', '2017_06.csv'), ('2017_04.csv', '2017_07.csv'), ('2017_05.csv', '2017_08.csv'), ('2017_07.csv', '2017_10.csv'), ('2017_08.csv', '2017_11.csv'), ('2017_10.csv', '2018_01.csv'), ('2017_11.csv', '2018_02.csv'), ('2017_12.csv', '2018_03.csv'), ('2018_01.csv', '2018_04.csv'), ('2018_02.csv', '2018_05.csv'), ('2018_03.csv', '2018_06.csv'), ('20

In [40]:
def fix_columns(df, desired_col):
    for col in desired_col:
        #print(col)
        df_col = df.loc[:, col]
        if df_col.dtype == 'object':
            if len(df_col) > 0:
                #print(col, df_col.iloc[0])                     
                if np.sum([x[:1] == "$" for x in df_col.iloc[:10]]) > 0:
                    df_col = df_col.fillna('$0.00')
                    new_col = [locale.atof(x[1:]) for x in df_col]
                    df.loc[:,col] = new_col
                elif df_col.iloc[0] == 'f' or df_col.iloc[0] == 't':
                    new_col = [(x=='t') for x in df_col]
                    df.loc[:,col] = new_col
        if col == "neighbourhood_cleansed" or col == "neighbourhood_group_cleansed":
            new_col = [str(x).lower() for x in df_col]
            df.loc[:,col] = new_col
    return df
def fix_zillow_regionname_countyname(df):
    for x in ["RegionName", "CountyName"]:
        df_col = df.loc[:,x]
        new_col = [str(x).lower() for x in df_col]
        df.loc[:,x] = new_col
    return df

#goes through all zillow data and airbnb dataframe for date pair, finds neighborhoods (neighborhood + county pair)
#that are in both data sets and counts how many listings for that neighborhood for each # bedrooms between 0 and 2
#returns list with tuple, where tuple contains matching neighborhood/county sets and a list of bools per bedroom number checking if there are enough listings 
def find_common_neigh(zillow_df, airbnb_df_prev, airbnb_df_curr):
    both_neighs = []
    airbnb_set_prev = set([(x,y) for x,y in zip(airbnb_df_prev["neighbourhood_cleansed"], airbnb_df_prev["neighbourhood_group_cleansed"])])
    airbnb_set_curr = set([(x,y) for x,y in zip(airbnb_df_curr["neighbourhood_cleansed"], airbnb_df_curr["neighbourhood_group_cleansed"])])
    zillow_set = set([(x,y) for x,y in zip(zillow_df["RegionName"],zillow_df["CountyName"])])
    for z, z_c in zillow_set:
        for a, a_c in airbnb_set_curr:
            if z.lower() == a.lower() and (z_c.lower() == a_c.lower() or a_c.lower() == 'nan'):
                for a_p, a_p_c in airbnb_set_prev:
                    if z.lower() == a_p.lower() and (z_c.lower() == a_p_c.lower() or a_p_c.lower() == 'nan'):
                        enough_pts_prev = [len(airbnb_df_prev.loc[(airbnb_df_prev['bedrooms'] == x)&(airbnb_df_prev['neighbourhood_cleansed'] == a)])>30 for x in range(3)]
                        enough_pts_curr = [len(airbnb_df_curr.loc[(airbnb_df_curr['bedrooms'] == x)&(airbnb_df_curr['neighbourhood_cleansed'] == a)])>30 for x in range(3)]
                        enough_pts = [(x&y) for x,y in zip(enough_pts_prev,enough_pts_curr)]
                        both_neighs.append((a.lower(), z_c.lower(), enough_pts))
    return both_neighs

def get_airbnb_county_from_lon_lat(df):
    pass

In [41]:
yr,prev_yr = read_in_airbnb_file_pair(lag_pairs[7], desired_col)
print(set(yr.loc[:,"neighbourhood_group_cleansed"]))

print(set(yr.loc[:,"room_type"]))

/Users/Kurtio/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (40,85) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


{'staten island', 'new york', 'queens', 'bronx', 'kings'}
{'Entire home/apt'}


In [222]:
zillow_files = pd.read_csv("Data/zillow_0_1_2_combined.csv")
zillow_files = fix_zillow_regionname_countyname(zillow_files)
#test_neighs = find_common_neigh(zillow_files, prev_yr, yr)                ###[mhsj] uncomment
test_neighs = find_common_neigh(zillow_files, all_years[1], all_years[2])  ###[mhsj] delete
print(np.sum([np.sum(neigh[2]) for neigh in test_neighs]))
zillow_columns = list(zillow_files.columns)
zillow_columns_fix = {}
for x in zillow_columns:
    new_x = x.replace('-', '_').replace('NumBeds', 'bedrooms')
    zillow_columns_fix[x] = new_x
zillow_files = zillow_files.rename(columns = zillow_columns_fix)
print(zillow_file
    s.columns)


74
Index(['Unnamed: 0', '2010_02', '2010_03', '2010_04', '2010_05', '2010_06',
       '2010_07', '2010_08', '2010_09', '2010_10',
       ...
       '2018_07', '2018_08', '2018_09', 'City', 'CountyName', 'Metro',
       'bedrooms', 'RegionName', 'SizeRank', 'State'],
      dtype='object', length=112)


In [137]:
desired_col_aggregate_quants =np.array(['beds', 'minimum_nights', 'availability_90','availability_365','review_scores_rating', 'review_scores_accuracy','review_scores_location', 'review_scores_value', 'calculated_host_listings_count', 'reviews_per_month'])




In [128]:
def get_num_same_listings(prev_yr, yr):
    pass
def get_rel_increase_in_listings(prev_yr, yr):
    pass
def get_rel_increase_in_avg_feats(prev_yr,yr,feats):
    pass
def get_rel_increase_in_adjprice(prev_yr,yr):
    pass

def create_labeled_set_regression(date_pairs, zillow_files, desired_col, desired_col_aggregate_quants, num_months_predict):
    all_pts = []
    for pair in date_pairs:
        prev_date = pair[0][:-4].split('_')
        date = pair[1][:-4].split('_')
        z_prev_date = [int(x) for x in prev_date]
        z_prev_date = [z_prev_date[0]+int((z_prev_date[1]+2)/12),(z_prev_date[1]+2)%12 + 1]
        z_date = [z_prev_date[0]+1, z_prev_date[1]]
        date = str(z_date[0])+"_"+str(z_date[1]).zfill(2)
        prev_date = str(z_prev_date[0])+"_"+str(z_prev_date[1]).zfill(2)
        print(date)
        if date in list(zillow_files.columns):
            print('found')
            yr, prev_yr = read_in_airbnb_file_pair(pair, zillow_files, desired_col)
            both_neighs = find_common_neigh(zillow_files, prev_yr, yr)
            for neigh in both_neighs:
                for bedrooms, enough in enumerate(neigh[2]):
                    if enough:
                        zillow_files_bd_neigh = zillow_files.loc[(zillow_files["bedrooms"]==bedrooms)&(zillow_files["RegionName"]==neigh[0])&(zillow_files["CountyName"]==neigh[1])]
                        zillow_files_bd_neigh = zillow_files_bd_neigh.loc[:, [prev_date, date]]
                        print(zillow_files_bd_neigh)
                        if len(zillow_files_bd_neigh) > 0:
                            enough_zillow_data = [x != 0 for x in zillow_files_bd_neigh.iloc[0]]
                            print(enough_zillow_data)
                            if np.sum(enough_zillow_data) >= 2:
                                print(np.sum(enough_zillow_data))
                                curr_features = [bedrooms]
                                yr_bd_neigh = yr.loc[(yr['bedrooms'] == bedrooms)&(yr['neighbourhood_cleansed'] == neigh[0])&(yr['neighbourhood_group_cleansed'] == neigh[1])]
                                prev_yr_bd_neigh = prev_yr.loc[(prev_yr['bedrooms'] == bedrooms)&(prev_yr['neighbourhood_cleansed'] == neigh[0])&(prev_yr['neighbourhood_group_cleansed'] == neigh[1])]





In [129]:
create_labeled_set_regression(airbnb_files_datePairs[-6:],zillow_files,desired_col,desired_col_aggregate_quants)

TypeError: create_labeled_set_regression() missing 1 required positional argument: 'num_months_predict'